## 📈 정상성과 인과성

시계열 데이터 분석의 핵심 개념인 **정상성(Stationarity)** 과 **그랜저 인과성(Granger Causality)** 을 다룹니다. 

---

## 1. 시계열 정상성(Stationarity)이란?

### 💡 개념 (Concept)

**정상성(Stationarity)** 이란 시계열 데이터의 통계적 특성(평균, 분산 등)이 시간의 흐름에 따라 변하지 않는다는 것을 의미합니다. 

마치 잔잔한 호수처럼, 데이터의 전반적인 모습이 어느 시점에서 보아도 일정하게 유지되는 상태를 말합니다.

* **왜 중요할까요?**
  
    미래를 예측하는 대부분의 시계열 모델은 **"데이터의 기본 규칙(패턴)이 과거나 지금이나 미래에도 동일할 것"이라고 가정**합니다. 데이터가 정상성을 띨 때, 우리는 과거의 패턴을 학습하여 미래를 더 안정적으로 예측할 수 있습니다. 반면, 데이터에 뚜렷한 **추세(Trend)** 나 시간에 따라 변하는 **변동성(Volatility)** 이 있다면, 이는 비정상(Non-stationary) 데이터이며 분석 전에 반드시 정상성을 만족하도록 변환해주어야 합니다.

* **정상성과 비정상성의 시각적 비교**
    * **정상 시계열**: 평균을 중심으로 일정한 폭 안에서 움직입니다.
    * **비정상 시계열**: 시간에 따라 평균이 계속 상승(추세)하거나, 변동 폭이 점점 커지거나 작아집니다.

### 💻 예시 코드 (Example Code)

아래 코드는 정상성을 띠는 데이터(백색소음)와 뚜렷한 추세를 가진 비정상 데이터를 생성하고 시각화하여 둘의 차이를 명확히 보여줍니다.

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

# 시각화를 위한 기본 설정
pd.options.plotting.backend = "plotly"

In [3]:
# 1. 정상 시계열 데이터 생성 (Standard Normal Distribution)
np.random.seed(42)
stationary_data = np.random.randn(200) # 
stationary_series = pd.Series(stationary_data, name="Stationary Data (White Noise)")

# 시각화
fig_stationary = px.line(stationary_series, title="✅ 정상 시계열의 예시")
fig_stationary.show()

In [4]:
# 2. 비정상 시계열 데이터 생성 (Random Walk with Drift)
trend = np.arange(200) * 0.2
non_stationary_data = np.random.randn(200).cumsum() + trend
non_stationary_series = pd.Series(non_stationary_data, name="Non-Stationary Data (Random Walk)")
# 시각화
fig_non_stationary = px.line(non_stationary_series, title="❌ 비정상 시계열의 예시 (추세 존재)")
fig_non_stationary.show()

### ✏️ 연습 문제 (Practice Problems)

1.  미국 월별 항공 승객 수(`air`) 데이터를 불러와 시각화하고, 이 데이터가 정상적으로 보이는지 혹은 비정상적으로 보이는지 이유와 함께 설명해보세요. (힌트: `px.data.air()`를 사용하면 쉽게 데이터를 로드할 수 있습니다.)

In [5]:
# 연습 문제 1번 풀이 공간
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
air_df = pd.read_csv(url, index_col=0, parse_dates=True)
air_df.head()

# fig_air = px.line(air_df, y="Passengers", title="월별 항공 승객 수")
# fig_air.show()
air_df.plot()

# 비정상 데이터

> 이 데이터는 비정상적(Non-stationary)으로 보입니다. 이유는,
> 1. 뚜렷한 상승 추세(Trend)가 관찰됩니다.
> 2. 계절적 패턴(Seasonality)이 반복적으로 나타납니다.
> 3. 시간이 지남에 따라 평균값이 지속적으로 증가합니다.
> 4. 변동성도 시간에 따라 변화하는 것으로 보입니다.



2.  `np.random.randn(300)`으로 무작위 데이터를 생성한 뒤, `.cumprod()` 메소드를 적용하여 새로운 시계열을 만드세요. 이 데이터는 정상성을 띨까요? 시각화하여 확인해보세요.

In [6]:
# 연습 문제 2번 풀이 공간
trend = np.random.randn(300) * 0.2 # 0.2 : noise 추가
non_stationary = np.random.randn(300).cumsum() + trend # np에도 cumsum 가능
pd.Series(non_stationary).plot(title="비정상 시계열")

(해석 작성)

---

## 2. 정상성 통계적 검정 (ADF & KPSS)

### 💡 개념 (Concept)

눈으로 데이터의 정상성을 판단하는 것은 주관적일 수 있습니다. 따라서 우리는 통계적인 가설 검정 방법을 사용하여 객관적으로 정상성을 진단합니다.

가장 널리 사용되는 두 가지 검정은 **ADF 검정**과 **KPSS 검정**입니다.

* **ADF 검정 (Augmented Dickey-Fuller Test)**
    * **귀무가설 (H0)**: "데이터가 비정상적이다 (단위근이 존재한다)."
    * **해석**: `p-value`가 유의수준(보통 0.05)보다 **작으면**, 귀무가설을 기각합니다. 즉, **"데이터가 정상적이다"** 라고 판단합니다.

* **KPSS 검정 (Kwiatkowski-Phillips-Schmidt-Shin Test)**
    * **귀무가설 (H0)**: "데이터가 (추세에 대해) 정상적이다."
    * **해석**: `p-value`가 유의수준(보통 0.05)보다 **크면**, 귀무가설을 기각하지 못합니다. 즉, **"데이터가 정상적이다"** 라고 판단합니다.

> **⚠️ 중요**: 두 검정은 귀무가설이 서로 반대이므로 해석에 주의해야 합니다. 보통 두 검정의 결과를 교차 확인하여 종합적으로 판단합니다.
>
> * ADF (p < 0.05) & KPSS (p > 0.05) → **정상성 (Stationary)**
> * ADF (p > 0.05) & KPSS (p < 0.05) → **비정상성 (Non-stationary)**

### 💻 예시 코드 (Example Code)

`statsmodels` 라이브러리를 사용하여 ADF와 KPSS 검정을 수행합니다. 실제 경제 데이터인 미국 월별 소매 판매 지수를 가져와 테스트해 보겠습니다.

In [7]:
!pip install statsmodels pandas-datareader

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import pandas_datareader.data as web
from statsmodels.tsa.stattools import adfuller, kpss
import pandas as pd

# FRED에서 미국 월별 소매 판매 데이터 가져오기 (2000-01-01 ~ 2023-12-31)
# RETAIL AND FOOD SERVICES SALES, TOTAL (MRTSSM44000USS)
df_retail = web.DataReader('MRTSSM44000USS', 'fred', start='2000-01-01', end='2023-12-31')
retail_sales = df_retail['MRTSSM44000USS']

# 데이터 시각화
fig = px.line(retail_sales, title="미국 월별 소매 판매 지수", labels={"value": "Sales Index", "index": "Date"})
fig.show()

# 비정상성을 보여줌

In [17]:
adfuller(retail_sales)
# np.float64(1.0744738670089387) = 통계량
# np.float64(0.9949927701976976) = pvalue

(np.float64(1.0744738670089387),
 np.float64(0.9949927701976976),
 11,
 276,
 {'1%': np.float64(-3.4542672521624214),
  '5%': np.float64(-2.87206958769775),
  '10%': np.float64(-2.5723807881747534)},
 np.float64(5597.211185382611))

In [16]:
# --- ADF 검정 함수 ---
def adf_test(series):
    result = adfuller(series.dropna()) # adfuller : adf 검정 함수
    print('* ADF Test Result')
    print(f'- ADF Statistic: {result[0]:.4f}')
    print(f'- p-value: {result[1]:.4f}')
    if result[1] < 0.05:
        print("결론: p-value < 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각)")
    else:
        print("결론: p-value >= 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각 실패)")

# --- KPSS 검정 함수 ---
def kpss_test(series):
    # 'c'는 constant(level)에 대한 정상성 검정
    result = kpss(series.dropna(), regression='c')
    print('* KPSS Test Result')
    print(f'- KPSS Statistic: {result[0]:.4f}')
    print(f'- p-value: {result[1]:.4f}')
    if result[1] < 0.05:
        print("결론: p-value < 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각)")
    else:
        print("결론: p-value >= 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각 실패)")

In [15]:
# ADF 검정 수행
adf_test(retail_sales)

* ADF Test Result
- ADF Statistic: 1.0745
- p-value: 0.9950
결론: p-value >= 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각 실패)


In [16]:
# KPSS 검정 수행
kpss_test(retail_sales)

* KPSS Test Result
- KPSS Statistic: 2.4178
- p-value: 0.0100
결론: p-value < 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각)


C:\Users\Admin\AppData\Local\Temp\ipykernel_14908\456275646.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




### ✏️ 연습 문제 (Practice Problems)

1.  FRED에서 **미국 산업 생산 지수(코드: `INDPRO`)** 데이터를 2010년부터 현재까지 가져오세요.
   * API 키 설정 필요 (https://fred.stlouisfed.org) : 회원가입후, 프로필 > API Key > Request API Key 클릭하여 발급받기
   * 발급받은 API 키를 환경변수 파일(.env)에 저장하세요. FRED_API_KEY=발급받은 API 키

In [9]:
!pip install fredapi python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import os
from dotenv import load_dotenv
from fredapi import Fred
load_dotenv()
fred = Fred(api_key=os.environ.get('FRED_API_KEY'))

In [19]:
indpro = fred.get_series('INDPRO', observation_start='2010-01-01')

# 가져온 데이터 확인 코드 작성
indpro.info()
indpro.plot()

<class 'pandas.core.series.Series'>
DatetimeIndex: 184 entries, 2010-01-01 to 2025-04-01
Series name: None
Non-Null Count  Dtype  
--------------  -----  
184 non-null    float64
dtypes: float64(1)
memory usage: 2.9 KB


2.  가져온 데이터를 시각화하고, ADF와 KPSS 검정을 모두 수행하여 정상성 여부를 판단하고 결과를 설명하세요.
   
    두 결과가 상반되게 나타났다면, 두 결과를 종합하여 판단합니다. 보통은 상반되게 나오면 비정상성으로 판단합니다.

In [18]:
# 연습 문제 2번 풀이 공간
adf_test(indpro)
print()
kpss_test(indpro)

* ADF Test Result
- ADF Statistic: -2.9097
- p-value: 0.0442
결론: p-value < 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각)

* KPSS Test Result
- KPSS Statistic: 0.8999
- p-value: 0.0100
결론: p-value < 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각)


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\3967353191.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




(정상성 판단 결과 해석 작성)

---

## 3. 비정상 데이터 정상화하기 (feat. 차분 & 로그 변환)

### 💡 개념 (Concept)

비정상 데이터를 정상 데이터로 변환하는 가장 일반적인 방법은 **차분(Differencing)** 과 **로그 변환(Log Transformation)** 입니다.

* **차분 (Differencing)**
    * **목적**: 데이터의 **추세(Trend)** 를 제거합니다.
    * **방법**: 현재 시점의 데이터에서 바로 이전 시점의 데이터를 빼줍니다. ($Y'_t = Y_t - Y_{t-1}$)
    * Pandas에서는 `.diff()` 메서드를 사용하여 간단하게 계산할 수 있습니다. 1차 차분으로 정상성을 만족하지 못하면, 2차 차분(`series.diff().diff()`)을 시도할 수 있습니다.

* **로그 변환 (Log Transformation)**
    * **목적**: 시간에 따라 변동성이 커지는 현상(분산의 불안정성)을 완화합니다.
    * **방법**: 데이터의 모든 값에 자연로그(`np.log()`)를 취합니다.
    * 보통 변동성과 추세가 함께 나타나는 데이터에 **로그 변환을 먼저 적용한 후, 차분을 수행**하는 경우가 많습니다.

### 💻 예시 코드 (Example Code)

앞서 사용한 비정상 데이터 '미국 월별 소매 판매 지수'를 정상 데이터로 변환해 보겠습니다. 이 데이터는 뚜렷한 추세와 함께 시간이 지날수록 변동성이 커지는 모습을 보이므로, 로그 변환 후 차분을 적용하는 것이 효과적입니다.

In [20]:
import numpy as np
import pandas as pd

# 1. 로그 변환 적용
log_retail_sales = np.log(retail_sales)
fig = px.line(log_retail_sales, title="1. 로그 변환 후 소매 판매 지수",
              labels={"value": "Log(Sales Index)", "index": "Date"})
fig.show()

In [21]:
# 2. 1차 차분 적용
diff_log_retail_sales = log_retail_sales.diff().dropna()
fig = px.line(diff_log_retail_sales, title="2. 로그 변환 및 1차 차분 후 소매 판매 지수",
              labels={"value": "Differenced Log Sales", "index": "Date"})
fig.show()

In [22]:
# 3. 변환된 데이터에 정상성 검정 재수행
print("--- 변환 후 데이터 정상성 검정 ---")
adf_test(diff_log_retail_sales)
kpss_test(diff_log_retail_sales)

--- 변환 후 데이터 정상성 검정 ---
* ADF Test Result
- ADF Statistic: -4.4743
- p-value: 0.0002
결론: p-value < 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각)
* KPSS Test Result
- KPSS Statistic: 0.1220
- p-value: 0.1000
결론: p-value >= 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각 실패)


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\3967353191.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




### ✏️ 연습 문제 (Practice Problems)

1.  **야후 파이낸스(`yfinance`)** 라이브러리를 사용하여 2015년부터 현재까지의 **애플(AAPL) 주가 데이터**를 가져오세요. 종가 데이터를 Series 변수에 저장합니다.


In [23]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [35]:
# 연습 문제 1번 풀이 공간
# '(Close, AAPL)' 컬럼 선택
import yfinance as yf

df = yf.download('AAPL', start='2015-01-01', end='2025-06-10')
df.info()

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2624 entries, 2015-01-02 to 2025-06-09
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   2624 non-null   float64
 1   (High, AAPL)    2624 non-null   float64
 2   (Low, AAPL)     2624 non-null   float64
 3   (Open, AAPL)    2624 non-null   float64
 4   (Volume, AAPL)  2624 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 123.0 KB


In [36]:
df.columns.levels
# multi Column -> [[]] : 중첩

FrozenList([['Close', 'High', 'Low', 'Open', 'Volume'], ['AAPL']])

In [37]:
# multi index 제거
df.columns = df.columns.levels[0]
df.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

2.  해당 주가 데이터가 비정상적임을 시각화와 통계 검정으로 확인하세요.

In [38]:
# 연습 문제 2번 풀이 공간
aapl_close = df['Close']
aapl_close.plot()

In [ ]:
# 2. 통계 검정 (ADF, KPSS)을 통한 비정상성 확인

# ADF (Augmented Dickey-Fuller) 검정
# 코드 작성
adf_test(aapl_close)
# KPSS (Kwiatkowski-Phillips-Schmidt-Shin) 검정
# 코드 작성
kpss_test(aapl_close)

* ADF Test Result
- ADF Statistic: -0.3578
- p-value: 0.9169
결론: p-value >= 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각 실패)

* KPSS Test Result
- KPSS Statistic: 8.0821
- p-value: 0.0100
결론: p-value < 0.05 이므로, 데이터는 비정상적입니다. (귀무가설 기각)


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\3967353191.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




3.  주가 데이터에 로그 변환과 차분을 순서대로 적용하고, 각 단계별로 데이터를 시각화하세요. 최종적으로 변환된 데이터가 정상성을 만족하는지 ADF와 KPSS 검정으로 확인하고 결과를 설명하세요.

In [47]:
# 연습 문제 3번 풀이 공간 (정상화)

# 1. 로그 변환
# 코드 작성
aapl_close_log_transformed = np.log(aapl_close)
aapl_close_log_transformed.plot()

# 2. 1차 차분
# 코드 작성
aapl_diff = aapl_close_log_transformed.dropna().diff()
aapl_diff.plot()

# 3. 각 단계별 데이터 시각화 (Plotly 사용)
# 코드 작성

# 4. 최종 변환된 데이터 (aapl_diff)의 정상성 검정 (ADF, KPSS)
# 코드 작성 및 해석 작성
adf_test(aapl_diff)
kpss_test(aapl_diff)


* ADF Test Result
- ADF Statistic: -16.3190
- p-value: 0.0000
결론: p-value < 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각)

* KPSS Test Result
- KPSS Statistic: 0.0846
- p-value: 0.1000
결론: p-value >= 0.05 이므로, 데이터는 정상적입니다. (귀무가설 기각 실패)


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\3967353191.py:15: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




---

## 4. 그랜저 인과성 (Granger Causality) 검정

### 💡 개념 (Concept)

**그랜저 인과성 검정**은 한 시계열 X의 과거 값이 다른 시계열 Y의 미래 값을 예측하는 데 유용한 정보를 제공하는지를 판단하는 통계적 가설 검정입니다.

* **핵심 질문**: "Y의 과거 정보만으로 Y를 예측하는 것보다, X의 과거 정보까지 함께 사용했을 때 Y의 예측력이 더 향상되는가?"
* 만약 예측력이 의미 있게 향상된다면, **"X가 Y를 그랜저-유발(Granger-causes)한다"**고 말합니다.
* **주의사항**:
    * 그랜저 인과성은 'A가 B의 원인이다'라는 철학적/실제적 인과관계를 의미하지 않습니다. 오직 **'예측에 도움이 되는가'**라는 관점의 통계적 관계입니다.
    * **매우 중요**: 그랜저 인과성 검정을 수행하기 전에, 분석에 사용될 **모든 시계열 데이터가 정상성을 만족**해야 합니다.

* **가설 검정**:
    * **귀무가설 (H0)**: "X가 Y를 그랜저-유발하지 않는다." (X의 과거 정보는 Y 예측에 도움이 되지 않는다)
    * **해석**: `p-value`가 유의수준(보통 0.05)보다 **작으면**, 귀무가설을 기각합니다. 즉, **"X가 Y를 그랜저-유발한다"**고 결론 내립니다.

    - 인과성 있다 != 그랜저-유발 -> 구분해서 사용

### 💻 예시 코드 (Example Code)

"소비자 물가지수(CPI)가 생산자 물가지수(PPI)에 영향을 줄까? 혹은 그 반대일까?" 라는 질문을 그랜저 인과성 검정으로 확인해보겠습니다.

In [48]:
# !pip install statsmodels pandas-datareader
import pandas_datareader.data as web
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd
import numpy as np

# FRED에서 월별 데이터 가져오기 (2000-01-01 ~ 2023-12-31)
# CPI (CPIAUCSL), PPI (PPIACO)
start = '2000-01-01'
end = '2023-12-31'
cpi = web.DataReader('CPIAUCSL', 'fred', start, end)
ppi = web.DataReader('PPIACO', 'fred', start, end)

# 두 데이터를 하나의 데이터프레임으로 합치기
df_prices = pd.concat([cpi, ppi], axis=1)
df_prices.columns = ['CPI', 'PPI']
df_prices.head()

,CPI,PPI
DATE,,
2000-01-01,169.3,128.3
2000-02-01,170.0,129.8
2000-03-01,171.0,130.8
2000-04-01,170.9,130.7
2000-05-01,171.2,131.6


In [49]:
# 1. 데이터 정상화 (로그 변환 + 1차 차분)
df_prices_transformed = np.log(df_prices).diff().dropna()

# 정상화된 데이터 시각화
fig = df_prices_transformed.plot(title="정상화된 CPI와 PPI (로그 수익률)", kind='line',
                                 labels={"value": "Log Return", "index": "Date"})
fig.show()

In [50]:
# 2. 그랜저 인과성 검정
# H0: PPI가 CPI를 그랜저-유발하지 않는다.
print("--- 검정 1: PPI -> CPI ---")
granger_test_result_1 = grangercausalitytests(df_prices_transformed[['CPI', 'PPI']], maxlag=4, verbose=False)
# p-value만 출력 (lag 1~4)
p_values_1 = [round(granger_test_result_1[lag+1][0]['ssr_ftest'][1], 4) for lag in range(4)]
print(f"Lag 1~4 p-values: {p_values_1}")
if any(p < 0.05 for p in p_values_1):
    print("결론: 하나 이상의 시차에서 p-value < 0.05 이므로, PPI가 CPI를 그랜저-유발한다고 볼 수 있습니다.")
else:
    print("결론: 모든 시차에서 p-value >= 0.05 이므로, PPI가 CPI를 그랜저-유발한다고 보기 어렵습니다.")

--- 검정 1: PPI -> CPI ---
Lag 1~4 p-values: [np.float64(0.0008), np.float64(0.0042), np.float64(0.0199), np.float64(0.042)]
결론: 하나 이상의 시차에서 p-value < 0.05 이므로, PPI가 CPI를 그랜저-유발한다고 볼 수 있습니다.


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results



In [53]:
granger_test_result_1[1]

({'ssr_ftest': (np.float64(11.37110597212105),
   np.float64(0.0008498918480583638),
   np.float64(283.0),
   np.int64(1)),
  'ssr_chi2test': (np.float64(11.491647731542828),
   np.float64(0.0006990963280297667),
   np.int64(1)),
  'lrtest': (np.float64(11.266780931053745),
   np.float64(0.0007890644172211322),
   np.int64(1)),
  'params_ftest': (np.float64(11.371105972120901),
   np.float64(0.0008498918480584086),
   np.float64(283.0),
   1.0)},
  array([[0., 1., 0.]])])

In [54]:
# p_values_1 = [round(granger_test_result_1[lag+1][0]['ssr_ftest'][1], 4) for lag in range(4)]
p_values_1 = []
for lag in range(4):
    p_val = round(granger_test_result_1[lag+1][0]['ssr_ftest'][1], 4) # [lag+1] : 정수로 가져와서 / 0부터 순회
    p_values_1.append(p_val)

p_values_1

[np.float64(0.0008), np.float64(0.0042), np.float64(0.0199), np.float64(0.042)]

In [55]:
# H0: CPI가 PPI를 그랜저-유발하지 않는다.
print("--- 검정 2: CPI -> PPI ---")
granger_test_result_2 = grangercausalitytests(df_prices_transformed[['PPI', 'CPI']], maxlag=4, verbose=False)
p_values_2 = [round(granger_test_result_2[lag+1][0]['ssr_ftest'][1], 4) for lag in range(4)]
print(f"Lag 1~4 p-values: {p_values_2}")
if any(p < 0.05 for p in p_values_2):
    print("결론: 하나 이상의 시차에서 p-value < 0.05 이므로, CPI가 PPI를 그랜저-유발한다고 볼 수 있습니다.")
else:
    print("결론: 모든 시차에서 p-value >= 0.05 이므로, CPI가 PPI를 그랜저-유발한다고 보기 어렵습니다.")

--- 검정 2: CPI -> PPI ---
Lag 1~4 p-values: [np.float64(0.8532), np.float64(0.0219), np.float64(0.0177), np.float64(0.0289)]
결론: 하나 이상의 시차에서 p-value < 0.05 이므로, CPI가 PPI를 그랜저-유발한다고 볼 수 있습니다.


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning:

verbose is deprecated since functions should not print results



### ✏️ 연습 문제 (Practice Problems)

1.  "M2 통화량(M2SL)"과 "실질 개인 소비 지출(PCECC96)"은 어떤 관계가 있을까요? 두 변수를 FRED에서 1990년부터 현재까지 월별 데이터로 가져오세요. (단, PCECC96은 월별 데이터가 아니므로, `M2SL`만 가져옵니다. 대신 `UNRATE`(실업률)을 사용해봅시다.)


In [56]:
# 연습 문제 1번 풀이 공간
# FRED에서 M2 통화량(M2SL)과 실업률(UNRATE) 데이터 가져오기
# 코드 작성
import yfinance as yf
import pandas_datareader.data as web
from datetime import datetime

# FRED에서 데이터 가져오기 (1990년부터 현재까지)
start_date = '1990-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

# M2 통화량과 실업률 데이터 가져오기
m2_data = web.get_data_fred('M2SL', start_date, end_date)  # M2 통화량
unrate_data = web.get_data_fred('UNRATE', start_date, end_date)  # 실업률

# 데이터 결합
df_macro = pd.concat([m2_data, unrate_data], axis=1)
df_macro.columns = ['M2SL', 'UNRATE']

# 결측값 제거
df_macro = df_macro.dropna()

print("M2 통화량과 실업률 데이터:")
print(df_macro.head())
print(f"\n데이터 기간: {df_macro.index[0]} ~ {df_macro.index[-1]}")
print(f"총 관측치 수: {len(df_macro)}")

M2 통화량과 실업률 데이터:
              M2SL  UNRATE
DATE                      
1990-01-01  3166.8     5.4
1990-02-01  3179.2     5.3
1990-03-01  3190.1     5.2
1990-04-01  3201.6     5.4
1990-05-01  3200.6     5.4

데이터 기간: 1990-01-01 00:00:00 ~ 2025-04-01 00:00:00
총 관측치 수: 424


2.  두 시계열(`M2SL`, `UNRATE`)을 각각 시각화하고, 로그 변환 + 차분을 통해 정상성을 만족하는 데이터로 변환하세요. (ADF/KPSS 검정으로 확인 필수)


In [57]:
# 연습 문제 2번 풀이 공간
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 시계열 시각화
# 코드 작성
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('M2 통화량 (M2SL)', '실업률 (UNRATE)'),
    vertical_spacing=0.1
)

# M2 통화량 그래프
fig.add_trace(
    go.Scatter(x=df_macro.index, y=df_macro['M2SL'], 
               name='M2SL', line=dict(color='blue')),
    row=1, col=1
)

# 실업률 그래프
fig.add_trace(
    go.Scatter(x=df_macro.index, y=df_macro['UNRATE'], 
               name='UNRATE', line=dict(color='red')),
    row=2, col=1
)

fig.update_layout(
    title='M2 통화량과 실업률 원본 데이터',
    height=600,
    showlegend=False
)

fig.update_xaxes(title_text="날짜", row=2, col=1)
fig.update_yaxes(title_text="M2SL (조 달러)", row=1, col=1)
fig.update_yaxes(title_text="UNRATE (%)", row=2, col=1)

fig.show()

# 로그 변환 (M2SL만, UNRATE는 이미 비율 데이터이므로 제외)
# 코드 작성
df_macro['log_M2SL'] = np.log(df_macro['M2SL'])

# 로그 변환된 M2SL 시각화
# 코드 작성
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_macro.index, y=df_macro['log_M2SL'], 
                         name='log(M2SL)', line=dict(color='green')))
fig2.update_layout(
    title='로그 변환된 M2 통화량',
    xaxis_title='날짜',
    yaxis_title='log(M2SL)'
)
fig2.show()

# ADF 검정 함수 정의
# 코드 작성
def perform_adf_test(series, name):
    result = adfuller(series.dropna())
    print(f"\n=== {name} ADF 검정 결과 ===")
    print(f"ADF 통계량: {result[0]:.6f}")
    print(f"p-value: {result[1]:.6f}")
    print("임계값:")
    for key, value in result[4].items():
        print(f"\t{key}: {value:.3f}")
    
    if result[1] <= 0.05:
        print("결론: 정상성을 만족합니다 (귀무가설 기각)")
    else:
        print("결론: 정상성을 만족하지 않습니다 (귀무가설 채택)")
    return result[1] <= 0.05

# KPSS 검정 함수 정의
# 코드 작성
def perform_kpss_test(series, name):
    result = kpss(series.dropna())
    print(f"\n=== {name} KPSS 검정 결과 ===")
    print(f"KPSS 통계량: {result[0]:.6f}")
    print(f"p-value: {result[1]:.6f}")
    print("임계값:")
    for key, value in result[3].items():
        print(f"\t{key}: {value:.3f}")
    
    if result[1] >= 0.05:
        print("결론: 정상성을 만족합니다 (귀무가설 채택)")
    else:
        print("결론: 정상성을 만족하지 않습니다 (귀무가설 기각)")
    return result[1] >= 0.05

# 원본 데이터 정상성 검정
# 코드 작성
print("=" * 50)
print("원본 데이터 정상성 검정")
print("=" * 50)

adf_m2sl = perform_adf_test(df_macro['M2SL'], 'M2SL')
kpss_m2sl = perform_kpss_test(df_macro['M2SL'], 'M2SL')

adf_log_m2sl = perform_adf_test(df_macro['log_M2SL'], 'log(M2SL)')
kpss_log_m2sl = perform_kpss_test(df_macro['log_M2SL'], 'log(M2SL)')

adf_unrate = perform_adf_test(df_macro['UNRATE'], 'UNRATE')
kpss_unrate = perform_kpss_test(df_macro['UNRATE'], 'UNRATE')

# 1차 차분 수행
# 코드 작성
df_macro['diff_log_M2SL'] = df_macro['log_M2SL'].diff()
df_macro['diff_UNRATE'] = df_macro['UNRATE'].diff()

# 차분된 데이터 시각화
# 코드 작성
fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=('1차 차분된 log(M2SL)', '1차 차분된 UNRATE'),
    vertical_spacing=0.1
)

fig3.add_trace(
    go.Scatter(x=df_macro.index, y=df_macro['diff_log_M2SL'], 
               name='diff_log_M2SL', line=dict(color='purple')),
    row=1, col=1
)

fig3.add_trace(
    go.Scatter(x=df_macro.index, y=df_macro['diff_UNRATE'], 
               name='diff_UNRATE', line=dict(color='orange')),
    row=2, col=1
)

fig3.update_layout(
    title='1차 차분된 데이터',
    height=600,
    showlegend=False
)

fig3.update_xaxes(title_text="날짜", row=2, col=1)
fig3.update_yaxes(title_text="diff_log_M2SL", row=1, col=1)
fig3.update_yaxes(title_text="diff_UNRATE", row=2, col=1)

fig3.show()

# 차분된 데이터 정상성 검정
# 코드 작성
print("\n" + "=" * 50)
print("차분된 데이터 정상성 검정")
print("=" * 50)

adf_diff_log_m2sl = perform_adf_test(df_macro['diff_log_M2SL'], '1차 차분된 log(M2SL)')
kpss_diff_log_m2sl = perform_kpss_test(df_macro['diff_log_M2SL'], '1차 차분된 log(M2SL)')

adf_diff_unrate = perform_adf_test(df_macro['diff_UNRATE'], '1차 차분된 UNRATE')
kpss_diff_unrate = perform_kpss_test(df_macro['diff_UNRATE'], '1차 차분된 UNRATE')

# 최종 정상성 데이터 준비
# 코드 작성
df_stationary = df_macro[['diff_log_M2SL', 'diff_UNRATE']].dropna()

print("\n" + "=" * 50)
print("정상성 변환 결과 요약")
print("=" * 50)
print("최종 분석용 데이터:")
print("- M2SL: 로그 변환 후 1차 차분")
print("- UNRATE: 1차 차분")
print(f"분석 가능한 관측치 수: {len(df_stationary)}")
print(f"데이터 기간: {df_stationary.index[0]} ~ {df_stationary.index[-1]}")

원본 데이터 정상성 검정

=== M2SL ADF 검정 결과 ===
ADF 통계량: 1.638470
p-value: 0.997969
임계값:
	1%: -3.447
	5%: -2.869
	10%: -2.571
결론: 정상성을 만족하지 않습니다 (귀무가설 채택)

=== M2SL KPSS 검정 결과 ===
KPSS 통계량: 3.037268
p-value: 0.010000
임계값:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739
결론: 정상성을 만족하지 않습니다 (귀무가설 기각)

=== log(M2SL) ADF 검정 결과 ===
ADF 통계량: 0.592108
p-value: 0.987419
임계값:
	1%: -3.446
	5%: -2.868
	10%: -2.570
결론: 정상성을 만족하지 않습니다 (귀무가설 채택)

=== log(M2SL) KPSS 검정 결과 ===
KPSS 통계량: 3.337356
p-value: 0.010000
임계값:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739
결론: 정상성을 만족하지 않습니다 (귀무가설 기각)

=== UNRATE ADF 검정 결과 ===
ADF 통계량: -2.679570
p-value: 0.077641
임계값:
	1%: -3.446
	5%: -2.868
	10%: -2.570
결론: 정상성을 만족하지 않습니다 (귀무가설 채택)

=== UNRATE KPSS 검정 결과 ===
KPSS 통계량: 0.277098
p-value: 0.100000
임계값:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739
결론: 정상성을 만족합니다 (귀무가설 채택)


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\2744006591.py:75: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\2744006591.py:75: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\2744006591.py:75: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.





차분된 데이터 정상성 검정

=== 1차 차분된 log(M2SL) ADF 검정 결과 ===
ADF 통계량: -5.611453
p-value: 0.000001
임계값:
	1%: -3.446
	5%: -2.868
	10%: -2.570
결론: 정상성을 만족합니다 (귀무가설 기각)

=== 1차 차분된 log(M2SL) KPSS 검정 결과 ===
KPSS 통계량: 0.265224
p-value: 0.100000
임계값:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739
결론: 정상성을 만족합니다 (귀무가설 채택)

=== 1차 차분된 UNRATE ADF 검정 결과 ===
ADF 통계량: -12.113836
p-value: 0.000000
임계값:
	1%: -3.446
	5%: -2.868
	10%: -2.570
결론: 정상성을 만족합니다 (귀무가설 기각)

=== 1차 차분된 UNRATE KPSS 검정 결과 ===
KPSS 통계량: 0.038951
p-value: 0.100000
임계값:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739
결론: 정상성을 만족합니다 (귀무가설 채택)

정상성 변환 결과 요약
최종 분석용 데이터:
- M2SL: 로그 변환 후 1차 차분
- UNRATE: 1차 차분
분석 가능한 관측치 수: 423
데이터 기간: 1990-02-01 00:00:00 ~ 2025-04-01 00:00:00


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\2744006591.py:75: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.


C:\Users\Admin\AppData\Local\Temp\ipykernel_6220\2744006591.py:75: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




(해석 작성)

3.  아래의 두 가지 가설에 대해 그랜저 인과성 검정을 수행하고(최대 시차 4), 각각의 결과를 해석하여 결론을 도출하세요.
    * 가설 1: M2 통화량이 실업률을 그랜저-유발하는가? (M2SL → UNRATE)
    * 가설 2: 실업률이 M2 통화량을 그랜저-유발하는가? (UNRATE → M2SL)

In [ ]:
# 연습 문제 3번 풀이 공간
# 그랜저 인과성 검정을 위한 라이브러리 import
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd

# 정상성을 만족하는 변환된 데이터 사용 (이전 단계에서 생성된 데이터)
# 로그 변환 후 1차 차분된 데이터
# 코드 작성
m2_stationary = np.log(df_macro['M2SL']).diff().dropna()
unrate_stationary = df_macro['UNRATE'].diff().dropna()

# 두 시계열의 인덱스를 맞춤
# 코드 작성
common_index = m2_stationary.index.intersection(unrate_stationary.index)
m2_aligned = m2_stationary.loc[common_index]
unrate_aligned = unrate_stationary.loc[common_index]

# 그랜저 인과성 검정용 데이터프레임 생성
# 코드 작성
granger_data = pd.DataFrame({
    'M2SL_diff': m2_aligned,
    'UNRATE_diff': unrate_aligned
})

print("=== 그랜저 인과성 검정 결과 ===\n")

# 가설 1: M2 통화량이 실업률을 그랜저-유발하는가? (M2SL → UNRATE)
# 코드 작성
print("가설 1: M2 통화량 → 실업률 (M2SL → UNRATE)")
print("-" * 50)

# 그랜저 인과성 검정 (최대 시차 4)
# 종속변수가 첫 번째 열, 독립변수가 두 번째 열
# 코드 작성
granger_test1 = grangercausalitytests(granger_data[['UNRATE_diff', 'M2SL_diff']], 
                                     maxlag=4, verbose=True)

print("\n" + "="*60 + "\n")

# 가설 2: 실업률이 M2 통화량을 그랜저-유발하는가? (UNRATE → M2SL)
# 코드 작성
print("가설 2: 실업률 → M2 통화량 (UNRATE → M2SL)")
print("-" * 50)

# 그랜저 인과성 검정 (최대 시차 4)
# 종속변수가 첫 번째 열, 독립변수가 두 번째 열
# 코드 작성
granger_test2 = grangercausalitytests(granger_data[['M2SL_diff', 'UNRATE_diff']], 
                                     maxlag=4, verbose=True)

print("\n" + "="*60)
print("=== 그랜저 인과성 검정 결과 해석 ===")
print("="*60)

# 결과 해석을 위한 p-value 추출 및 정리
# 코드 작성
print("\n1. M2 통화량 → 실업률 (M2SL → UNRATE) 검정 결과:")
for lag in range(1, 5):
    f_stat = granger_test1[lag][0]['ssr_ftest'][0]
    p_value = granger_test1[lag][0]['ssr_ftest'][1]
    print(f"   시차 {lag}: F-통계량 = {f_stat:.4f}, p-value = {p_value:.4f}")
    if p_value < 0.05:
        print(f"   → 시차 {lag}에서 유의수준 5%로 그랜저 인과관계 존재")
    else:
        print(f"   → 시차 {lag}에서 그랜저 인과관계 없음")

print("\n2. 실업률 → M2 통화량 (UNRATE → M2SL) 검정 결과:")
for lag in range(1, 5):
    f_stat = granger_test2[lag][0]['ssr_ftest'][0]
    p_value = granger_test2[lag][0]['ssr_ftest'][1]
    print(f"   시차 {lag}: F-통계량 = {f_stat:.4f}, p-value = {p_value:.4f}")
    if p_value < 0.05:
        print(f"   → 시차 {lag}에서 유의수준 5%로 그랜저 인과관계 존재")
    else:
        print(f"   → 시차 {lag}에서 그랜저 인과관계 없음")

#### 그랜저 인과성 검정 결과 해석
- **귀무가설**: X가 Y를 그랜저-유발하지 않는다
- **대립가설**: X가 Y를 그랜저-유발한다
- **판단기준**: 유의수준 5% 기준으로 p-value < 0.05이면 귀무가설 기각

#### 경제학적 해석
- M2 통화량의 변화가 실업률 변화를 예측하는 데 도움이 되는지 분석
- 실업률의 변화가 M2 통화량 변화를 예측하는 데 도움이 되는지 분석
- 통화정책과 노동시장 간의 상호작용을 이해하는 데 중요한 정보 제공

#### 실무적 시사점
- 중앙은행의 통화정책 결정 시 노동시장 상황 고려 필요성
- 경제정책 수립 시 통화량과 고용 간의 시차 효과 반영
- 거시경제 예측 모델에서 변수 간 인과관계 활용 가능
